# 1. Introduction
This notebook contains example workflow for web services exposed by the EIDA (European Integrated Data Archive) Federation, focused on the EIDAWS-WFCatalog service and its functionalities allowing users to filter out low quality and low coverage data before downloading it to the local machine.

In this example we are going to:

1. Retrieve seismic event information from FDSNWS-Event catalogue offered by [GFZ](https://www.gfz-potsdam.de/) EIDA Node:
    * Start date = 2020-01-01
    * End date = 2020-06-01
    * Minimum magnitude = 5
    * Minimum latitude = 40
    * Maximum latitude = 45
    * Minimum longitude = 17
    * Maximum longitude = 25
1. List all stations available in proximity to given event using [FDSNWS-Station](https://www.orfeus-eu.org/data/eida/webservices/station/) web service
1. Using [EIDAWS-WFCatalog](https://www.orfeus-eu.org/data/eida/webservices/wfcatalog/) service, we are going to exclude all stations which do not meet following criteria:
    * At least 95% data coverage on the day of the event
    * Maximum of 5 gaps
    * Sum of gaps lower than 50 seconds
    * No overlaps
1. Using [FDSNWS-Dataselect](https://www.orfeus-eu.org/data/eida/webservices/dataselect/) web service, we are going to download:
    * miniSEED file containing the waveforms
    * StationXML file containing station and instrumentation metadata on channel level
1. At the end we are going to quickly demo the GUI-based tools based on the [EIDAWS-WFCatalog](https://www.orfeus-eu.org/data/eida/webservices/wfcatalog/) web service, available via [data quality page](https://www.orfeus-eu.org/data/eida/quality/) on the [orfeus-eu.org](https://orfeus-eu.org).

For downloading data from FDSNWS-Station and FDSNWS-Dataselect (and EIDAWS-WFCatalog) services we are going to directly use [EIDAWS-Federator](http://eida-federator.ethz.ch/) gateway. This means that data will be requested using:

```python
read_events(
    pathname_or_url=None,
    format=None,
    **kwargs
)
```

and

```python
read(
    pathname_or_url=None,
    format=None,
    headonly=False,
    starttime=None,
    endtime=None,
    nearest_sample=True,
    dtype=None,
    apply_calib=False,
    check_compression=True,
    **kwargs
)
```

rather than using the `RoutingClient` utilizing the [EIDAWS-Routing](https://www.orfeus-eu.org/data/eida/webservices/routing/):

```python
from obspy.clients.fdsn import RoutingClient
from obspy import UTCDateTime

rsClient = RoutingClient("eida-routing")
st = rsClient.get_waveforms(
    network="Z3",
    channel="HHZ",
    starttime=UTCDateTime(2016, 3, 1),
    endtime=UTCDateTime(2016, 3, 1, 0, 2, 0)
)
```

Please keep in mind that [EIDAWS-Federator](http://eida-federator.ethz.ch/) only provides acces to open data, so if your intention is to download restricted datasets, please refer to [EIDA/userfeedback](https://github.com/EIDA/userfeedback) GitHub repository for instructions.

# 2. Finding event
Lets start by finding an event. Below two completely different approaches have been presented:
* Downloading event XML document directly from FDSNWS-Event web service
* Downloading event information using ObsPy library


In [ ]:
import requests
import xml.etree.ElementTree as ET

ENCODING = "utf-8"
NSMAP = {'mw': 'http://quakeml.org/xmlns/bed/1.2'}

# Define start and end dates
start = "2020-01-01"
end = "2020-06-01"

# Define the bounding box
min_mag = 5
min_lat = 40
max_lat = 45
min_lon = 17
max_lon = 25

# Build URL to retrieve event information from event catalogue hosted by GFZ
events_url = (
    f"http://geofon.gfz-potsdam.de/fdsnws/event/1/query?"
    f"start={start}&end={end}&"
    f"minmag={min_mag}&"
    f"minlat={min_lat}&minlon={min_lon}&"
    f"maxlat={max_lat}&maxlon={max_lon}"
)

# http://geofon.gfz-potsdam.de/fdsnws/event/1/query?start=2020-01-01&end=2020-06-01&minmag=5&minlat=40&minlon=17&maxlat=45&maxlon=25

r = requests.get(events_url, timeout=10)
r.encoding = ENCODING

if r.status_code == 200:
    event_root = ET.fromstring(r.text)
    for event_element in event_root.findall("./mw:eventParameters/*", namespaces=NSMAP):
        # Get event ID from event_element attribute
        event_id = event_element.get("publicID")
        
        # Find event magnitude element
        event_magnitude_element = event_element.find("./mw:magnitude/mw:mag/mw:value", namespaces=NSMAP)
        
        # Get magnitude value from event_magnitude_element
        event_magnitude = event_magnitude_element.text
        
        # Get event origin time element
        event_origin_time_element = event_element.find("./mw:origin/mw:time/mw:value", namespaces=NSMAP)
        
        # Get origin time value from event_origin_time_element
        event_origin_time = event_origin_time_element.text
        
        print(f"Event with ID: {event_id} happened on {event_origin_time} and had magnitude equal to: {event_magnitude}")
else:
    print("Something went wrong...")

And now with ObsPy...

In [ ]:
from obspy import read_events

# Read events using the same URL
evts = read_events(events_url)

# Print retrieved events
for e in evts:
    print(e)

Users are free to choose how they download data from EIDA, but for convenience we will continue using ObsPy throughout the rest of this notebook. 🦥

# 3. Finding stations

In [ ]:
from obspy import read_inventory

# Build URL using EIDA Federator and our defined parameters
stations_url = (
    f"http://eida-federator.ethz.ch/fdsnws/station/1/query?"
    f"start={start}&end={end}&"
    f"minlat={min_lat}&minlon={min_lon}&"
    f"maxlat={max_lat}&maxlon={max_lon}"
)

inv = read_inventory(stations_url)

print(inv)

Let's try to visualize our event and surrounding stations. Please notice that stations in this area are hosted by various EIDA nodes:
- NOA
- NIEP
- GFZ
- INGV
- ODC

![](img/eventAndStations.png)

# 4. Filter stations which do not meet our quality criteria
We want to download data from stations which have:
1. At least 95% data coverage on the day of the event
1. Maximum of 5 gaps
1. Sum of gaps lower than 50 seconds
1. No overlaps

First, lets see what default EIDAWS-WFCatalog response offers...

http://eida-federator.ethz.ch/eidaws/wfcatalog/1/query?start=2020-01-28T20:10:10.670309&end=2020-01-28T20:30:10.670309&network=HL&station=KZN

```json
[
  {
    "version": "1.0.0",
    "producer": {
      "name": "EIDA NODE",
      "agent": "ObsPy mSEED-QC",
      "created": "2020-01-29T08:39:49.718Z"
    },
    "station": "KZN",
    "network": "HL",
    "location": "",
    "channel": "HHZ",
    "num_gaps": 3,
    "num_overlaps": 0,
    "sum_gaps": 5,
    "sum_overlaps": 0,
    "max_gap": 2,
    "max_overlap": null,
    "record_length": [512],
    "sample_rate": [100],
    "percent_availability": 99.99421296296296,
    "encoding": ["STEIM2"],
    "num_records": 16260,
    "start_time": "2020-01-28T00:00:00.000Z",
    "end_time": "2020-01-29T00:00:00.000Z",
    "format": "miniSEED",
    "quality": "D"
  },
  {
    "version": "1.0.0",
    "producer": {
      "name": "EIDA NODE",
      "agent": "ObsPy mSEED-QC",
      "created": "2020-01-29T08:40:43.578Z"
    },
    "station": "KZN",
    "network": "HL",
    "location": "",
    "channel": "HHE",
    "num_gaps": 3,
    "num_overlaps": 0,
    "sum_gaps": 5,
    "sum_overlaps": 0,
    "max_gap": 2,
    "max_overlap": null,
    "record_length": [512],
    "sample_rate": [100],
    "percent_availability": 99.99421296296296,
    "encoding": ["STEIM2"],
    "num_records": 18823,
    "start_time": "2020-01-28T00:00:00.000Z",
    "end_time": "2020-01-29T00:00:00.000Z",
    "format": "miniSEED",
    "quality": "D"
  },
  {
    "version": "1.0.0",
    "producer": {
      "name": "EIDA NODE",
      "agent": "ObsPy mSEED-QC",
      "created": "2020-01-29T08:16:35.906Z"
    },
    "station": "KZN",
    "network": "HL",
    "location": "",
    "channel": "HHN",
    "num_gaps": 3,
    "num_overlaps": 0,
    "sum_gaps": 5,
    "sum_overlaps": 0,
    "max_gap": 2,
    "max_overlap": null,
    "record_length": [512],
    "sample_rate": [100],
    "percent_availability": 99.99421296296296,
    "encoding": ["STEIM2"],
    "num_records": 17616,
    "start_time": "2020-01-28T00:00:00.000Z",
    "end_time": "2020-01-29T00:00:00.000Z",
    "format": "miniSEED",
    "quality": "D"
  }
]
```

In [ ]:
import json
from datetime import datetime, timedelta

# Convert string representation of origin time to datetime object
dt_origin = datetime.strptime(event_origin_time, "%Y-%m-%dT%H:%M:%S.%fZ")

# Define relative time windows using event origin time
dt_start = dt_origin - timedelta(minutes=5)
dt_end = dt_origin + timedelta(minutes=15)

# https://federator.orfeus-eu.org/eidaws/wfcatalog/1/query?starttime=2020-01-28T20:14:10.670309&endtime=2020-01-28T20:16:10.670309&network=HL&station=KZN

# Channels we are interested in
# channels will be downloaded only if all specified components are present and validated
required_channels = {
    "BH?": ["BHE", "BHN", "BHZ"],
    "HH?": ["HHE", "HHN", "HHZ"],
}

# Our validating function
def validate_station(string):
    # Parse string to json object
    j = json.loads(string)
    
    channels_found = []
    channels_validated = []

    for cha in j:
        # Get network station channel identifiers
        network_code = cha["network"]
        station_code = cha["station"]
        channel_code = cha["channel"]
        
        # Get the quality metrics
        availability = int(cha["percent_availability"])
        gaps = int(cha["num_gaps"])
        sum_gaps = int(cha["sum_gaps"])
        overlaps = int(cha["num_overlaps"])
        
        msg = f"{network_code}.{station_code}.{channel_code}: {availability}% coverage, {gaps} gaps ({sum_gaps}s), {overlaps} overlaps."
        if availability < 95 or gaps > 5 or sum_gaps > 50 or overlaps > 0:
            print(f"NOK: {msg}")
        else:
            print(f"OK: {msg}")
            channels_found.append(channel_code.upper())

    # If channel components are present and validated, add them to the channels_validated list
    for c in required_channels.keys():
        if all(e in channels_found for e in required_channels[c]):
            channels_validated.append(c)

    if len(channels_validated) > 0:
        return ",".join(channels_validated)
    else:
        return None
    
# List of verified stations, will be filled later
verified_stations = []

# Loop through all networks and stations found in the bounding box surrounding our event
for net in inv:
    for sta in net:
        # Build WFCatalog URL (POST method is not avaiable via Federator)
        wfcatalog_url = (
            f"http://eida-federator.ethz.ch/eidaws/wfcatalog/1/query?"
            f"start={dt_start.isoformat()}&end={dt_end.isoformat()}&"
            f"network={net.code}&station={sta.code}"
        )

        # Request the data...
        r = requests.get(wfcatalog_url, timeout=10)
        r.encoding = ENCODING
        
        if r.status_code == 200:
            # Validate station
            cha = validate_station(r.text)
            if cha:
                # Validation passed, add to verified_stations list
                verified_stations.append([net.code, sta.code, cha])
        else:
            # print(f"No data for {net.code}.{sta.code}")
            pass

print("The verified stations are:", verified_stations)



# 5. Download data for verified stations

In [ ]:
from obspy import read

# Loop through our validated stations, build URL, retrieve and read the data
for s in verified_stations:
    dataselect_url = (
        f"http://eida-federator.ethz.ch/fdsnws/dataselect/1/query?"
        f"start={dt_start.isoformat()}&end={dt_end.isoformat()}&"
        f"network={s[0]}&station={s[1]}&"
        f"channel={s[2]}"
    )
    print(dataselect_url)
    st = read(dataselect_url)
    
    # Print simple waveform metadata and plot the waveforms
    print(st)
    st.plot()

# 6. Quality tools on the orfeus-eu.org
Overview of the EIDA Quality Tools: https://www.orfeus-eu.org/data/eida/quality/

## 6.1. Availability
Availability information can be easily previewed using web tool accessible via https://www.orfeus-eu.org/data/eida/quality/availability/:
![](img/availability.png)

## 6.2. Data Quality Inspector
Data quality inspector rendering advanced waveform metrics is avaiable via https://www.orfeus-eu.org/data/eida/quality/metrics/.

Following statistical parameters can be selected and rendered:
* Quadratic mean
* Standard deviation
* Minimum
* Maximum
* Availability
* Gaps
* Sum of gaps
* Overlaps
* Sum of overlaps
* Median
* Mean
* Lower quartile
* Upper quartile

![](img/inspector.png)